In [2]:
pip install spuco --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: spuco
    Found existing installation: spuco 2.0.2
    Uninstalling spuco-2.0.2:
      Successfully uninstalled spuco-2.0.2


In [3]:
import os
import torch
import pandas as pd
import torchvision.transforms as transforms
from torch.optim import SGD
from wilds import get_dataset

from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
from spuco.evaluate import Evaluator
from spuco.group_inference import SpareInference
from spuco.robust_train import SpareTrain
from spuco.models import model_factory
from spuco.utils import Trainer, set_seed

In [4]:
params = {
    "gpu": 0,
    "seed": 0,
    "root_dir": "/data",
    "batch_size": 32,
    "num_epochs": 20,
    "lr": 1e-3,
    "weight_decay": 1e-2,
    "momentum": 0.9,
    "pretrained": False,
    "infer_lr": 1e-3,
    "infer_weight_decay": 1e-2,
    "infer_momentum": 0.9,
    "infer_num_epochs": 1,
    "high_sampling_power": 2,
}

In [5]:
device = torch.device(f"cuda:{params['gpu']}" if torch.cuda.is_available() else "cpu")
set_seed(params["seed"])

In [6]:
classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root=f"{params['root_dir']}/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

valset = SpuCoMNIST(
    root=f"{params['root_dir']}/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val",
)
valset.initialize()

testset = SpuCoMNIST(
    root=f"{params['root_dir']}/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


In [7]:
model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes, pretrained=params["pretrained"]).to(device)

trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=params["batch_size"],
    optimizer=SGD(model.parameters(), lr=params["infer_lr"], weight_decay=params["infer_weight_decay"], momentum=params["infer_momentum"]),
    device=device,
    verbose=True
)

trainer.train(num_epochs=params["infer_num_epochs"])

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 0:   1%|          | 10/1501 [00:01<03:05,  8.06batch/s, accuracy=25.0%, loss=1.61]  

 | Epoch 0 | Loss: 1.6115756034851074 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6203930377960205 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.61808180809021 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6209357976913452 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6147788763046265 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6116019487380981 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6081494092941284 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.607947826385498 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.616422414779663 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6171921491622925 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.618257761001587 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.618667483329773 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.6032153367996216 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.60996413230896 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6192119121551514 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6320838928222656 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.6056013107299805 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6140894889831

Epoch 0:   3%|▎         | 40/1501 [00:01<00:36, 39.54batch/s, accuracy=43.75%, loss=1.6]

 | Epoch 0 | Loss: 1.5921324491500854 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6107178926467896 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.608851671218872 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6068270206451416 | Accuracy: 37.5%
 | Epoch 0 | Loss: 1.6245592832565308 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6247491836547852 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6223483085632324 | Accuracy: 28.125%
 | Epoch 0 | Loss: 1.5961277484893799 | Accuracy: 59.375%
 | Epoch 0 | Loss: 1.5983879566192627 | Accuracy: 40.625%
 | Epoch 0 | Loss: 1.610825777053833 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6113169193267822 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6118828058242798 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6016209125518799 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6103163957595825 | Accuracy: 34.375%
 | Epoch 0 | Loss: 1.5956121683120728 | Accuracy: 37.5%
 | Epoch 0 | Loss: 1.613802433013916 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6160727739334106 | Accuracy: 28.125%
 | Epoch 0 | Loss: 1.6188437

Epoch 0:   5%|▍         | 68/1501 [00:02<00:20, 70.50batch/s, accuracy=40.625%, loss=1.59]

...

Epoch 0: 100%|█████████▉| 1494/1501 [00:15<00:00, 85.76batch/s, accuracy=96.875%, loss=0.169] 

 | Epoch 0 | Loss: 0.011725349351763725 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.007039968855679035 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.2041069120168686 | Accuracy: 96.875%
 | Epoch 0 | Loss: 0.007489541079849005 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.0066990298219025135 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.007772465236485004 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.007507049944251776 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.008330882526934147 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.18133826553821564 | Accuracy: 96.875%
 | Epoch 0 | Loss: 0.008947423659265041 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.007181425113230944 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.6844380497932434 | Accuracy: 93.75%
 | Epoch 0 | Loss: 0.00842180848121643 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.007551389280706644 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.008337300270795822 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.007931418716907501 | Accuracy: 100.0%
 | Epoch 0 | Loss: 0.008958746679127216 | Accuracy: 100.0%


Epoch 0: 100%|█████████▉| 1494/1501 [00:15<00:00, 85.76batch/s, accuracy=100.0%, loss=0.00745]

 | Epoch 0 | Loss: 0.16851384937763214 | Accuracy: 96.875%
 | Epoch 0 | Loss: 0.007449684664607048 | Accuracy: 100.0%


Epoch 0: 100%|██████████| 1501/1501 [00:16<00:00, 92.72batch/s, accuracy=100.0%, loss=0.00745]


In [9]:
logits = trainer.get_trainset_outputs()
predictions = torch.nn.functional.softmax(logits, dim=1)

spare_infer = SpareInference(
    logits=predictions,
    class_labels=trainset.labels,
    device=device,
    max_clusters=5,
    high_sampling_power=params["high_sampling_power"],
    verbose=True
)

group_partition = spare_infer.infer_groups()
print(group_partition)
sampling_powers = spare_infer.sampling_powers

print("Sampling powers:", sampling_powers)
for key in sorted(group_partition.keys()):
    for true_key in sorted(trainset.group_partition.keys()):
        print(f"Inferred group: {key}, true group: {true_key}, size: {len([x for x in trainset.group_partition[true_key] if x in group_partition[key]])}")


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Clustering class-wise:   0%|          | 0/5 [00:00<?, ?it/s]

For n_clusters = 2 The average silhouette_score is : 0.9946167
For n_clusters = 3 The average silhouette_score is : 0.9962789
For n_clusters = 4 The average silhouette_score is : 0.9974368


Clustering class-wise:  20%|██        | 1/5 [00:06<00:25,  6.41s/it]

For n_clusters = 5 The average silhouette_score is : 0.99831325
Silhouette score for class 2: 0.9983132481575012
For n_clusters = 2 The average silhouette_score is : 0.98960465
For n_clusters = 3 The average silhouette_score is : 0.9910108
For n_clusters = 4 The average silhouette_score is : 0.9923443


Clustering class-wise:  40%|████      | 2/5 [00:12<00:19,  6.47s/it]

...

Clustering class-wise:  80%|████████  | 4/5 [00:22<00:05,  5.50s/it]

For n_clusters = 5 The average silhouette_score is : 0.9978694
Silhouette score for class 1: 0.9978693723678589
For n_clusters = 2 The average silhouette_score is : 0.9919296
For n_clusters = 3 The average silhouette_score is : 0.993502
For n_clusters = 4 The average silhouette_score is : 0.9946918


Clustering class-wise: 100%|██████████| 5/5 [00:29<00:00,  5.90s/it]

For n_clusters = 5 The average silhouette_score is : 0.99562186
Silhouette score for class 3: 0.9956218600273132
{(2, 0): [0, 2, 9, 10, 17, 23, 32, 44, 50, 55, 57, 58, 77, 83, 95, 107, 111, 115, 118, 123, 133, 134, 136, 142, 144, 149, 157, 159, 176, 178, 189, 190, 192, 197, 207, 212, 215, 216, 217, 218, 220, 225, 228, 230, 232, 233, 236, 245, 248, 250, 263, 266, 267, 271, 287, 292, 295, 296, 299, 301, 302, 314, 319, 320, 326, 327, 328, 331, 334, 347, 348, 364, 366, 389, 394, 402, 403, 404, 405, 423, 428, 429, 448, 454, 462, 467, 482, 483, 486, 488, 490, 494, 505, 515, 519, 520, 526, 528, 542, 543, 544, 546, 551, 553, 558, 559, 560, 563, 572, 573, 574, 577, 581, 591, 593, 595, 596, 601, 612, 616, 619, 637, 639, 643, 644, 653, 665, 669, 670, 671, 672, 677, 680, 682, 683, 684, 685, 688, 704, 716, 719, 725, 731, 733, 734, 738, 742, 756, 758, 766, 769, 773, 776, 779, 792, 794, 795, 797, 806, 808, 823, 830, 834, 836, 846, 856, 864, 868, 875, 877, 880, 884, 885, 890, 894, 895, 896, 915, 917, 

Inferred group: (0, 0), true group: (0, 0), size: 10082
Inferred group: (0, 0), true group: (0, 1), size: 0
Inferred group: (0, 0), true group: (0, 2), size: 0
Inferred group: (0, 0), true group: (0, 3), size: 0
Inferred group: (0, 0), true group: (0, 4), size: 0
Inferred group: (0, 0), true group: (1, 0), size: 0
Inferred group: (0, 0), true group: (1, 1), size: 0
Inferred group: (0, 0), true group: (1, 2), size: 0
Inferred group: (0, 0), true group: (1, 3), size: 0
Inferred group: (0, 0), true group: (1, 4), size: 0
Inferred group: (0, 0), true group: (2, 0), size: 0
Inferred group: (0, 0), true group: (2, 1), size: 0
Inferred group: (0, 0), true group: (2, 2), size: 0
Inferred group: (0, 0), true group: (2, 3), size: 0
Inferred group: (0, 0), true group: (2, 4), size: 0
Inferred group: (0, 0), true group: (3, 0), size: 0
Inferred group: (0, 0), true group: (3, 1), size: 0
Inferred group: (0, 0), true group: (3, 2), size: 0
Inferred group: (0, 0), true group: (3, 3), size: 0
Inferred

In [10]:
valid_evaluator = Evaluator(
    testset=valset,
    group_partition=valset.group_partition,
    group_weights=valset.group_weights,
    batch_size=params["batch_size"],
    model=model,
    device=device,
    verbose=True
)

spare_train = SpareTrain(
    model=model,
    num_epochs=params["num_epochs"],
    trainset=trainset,
    group_partition=group_partition,
    sampling_powers=[20] * 5,
    batch_size=params["batch_size"],
    optimizer=SGD(model.parameters(), lr=params["lr"], weight_decay=params["weight_decay"], momentum=params["momentum"]),
    device=device,
    val_evaluator=valid_evaluator,
    verbose=True
)
spare_train.train()

Output hidden; open in https://colab.research.google.com to view.

In [12]:
evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=params["batch_size"],
    model=spare_train.best_model,
    device=device,
    verbose=True
)
evaluator.evaluate()

print("Final Results:")
print(f"Worst Group Accuracy: {evaluator.worst_group_accuracy[1]}")
print(f"Average Accuracy: {evaluator.average_accuracy}")

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:08,  2.85it/s]

Group (0, 0) Accuracy: 0.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:08,  2.82it/s]

Group (0, 1) Accuracy: 59.57446808510638


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:07,  2.78it/s]

...

Group (4, 2) Accuracy: 0.0


Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:09<00:00,  2.33it/s]

Group (4, 3) Accuracy: 99.4949494949495


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:09<00:00,  2.53it/s]

Group (4, 4) Accuracy: 0.0
Final Results:
Worst Group Accuracy: 0.0
Average Accuracy: 0.42631064629556487
